In [110]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

### Importing Stocks

In [111]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


### Aquire API token

In [112]:
from secrets import IEX_CLOUD_API_TOKEN

In [113]:
symbol = 'AAPL'
# covert to f string
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 124828574, 'calculationPrice': 'tops', 'change': 2.138, 'changePercent': 0.01222, 'close': 0, 'closeSource': 'olacfiif', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': 0, 'highSource': 'Ic trpemE r Xiaelei', 'highTime': 1716369318244, 'iexAskPrice': 178.04, 'iexAskSize': 105, 'iexBidPrice': 178.67, 'iexBidSize': 103, 'iexClose': 182.783, 'iexCloseTime': 1676774133274, 'iexLastUpdated': 1680697338201, 'iexMarketPercent': 0.009467661295796488, 'iexOpen': 173.94, 'iexOpenTime': 1650416757474, 'iexRealtimePrice': 177.229, 'iexRealtimeSize': 104, 'iexVolume': 664730, 'lastTradeTime': 1685978841283, 'latestPrice': 177.147, 'latestSource': 'IEX real time price', 'latestTime': '3:43:06 PM', 'latestUpdate': 1698959759549, 'latestVolume': None, 'low': 0, 'lowSource': None, 'lowTime': None, 'marketCap

In [114]:
price = data['latestPrice']
market_cap = data['marketCap']
price

177.147

### Add our stocks to a dataframe

In [115]:
# add our columns
columns = ['Ticker', 'Stocks Price', 'Market Cap', 'Number of Shares to buy']
df = pd.DataFrame(columns=columns)


In [116]:
# add 1 row of data
df.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        'n/a',
    ], index = columns)
    
    ,
    ignore_index=True
)

,Ticker,Stocks Price,Market Cap,Number of Shares to buy
0,AAPL,177.147,2948989103779,n/a


In [117]:
# do it for all data
df = pd.DataFrame(columns = columns)
for stock in stocks['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    df = df.append( pd.Series(
    [
        stock,
        data['latestPrice'],
        data['marketCap'],
        'n/a',
    ], index = columns)
    ,ignore_index=True)

In [118]:
df

,Ticker,Stocks Price,Market Cap,Number of Shares to buy
0,A,159.170,48528716616,n/a
1,AAL,19.099,12131906601,n/a
2,AAP,236.351,14593314785,n/a
3,AAPL,175.600,2879694635479,n/a
4,ABBV,136.450,239662630756,n/a


## Use Batch API calls to improve performance

In [119]:
# split our list in 5 parts because there is a limit in batch api calls 100
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [120]:
groups = list(chunks(stocks['Ticker'], 100))
strs = []
for i in range(0, len(groups)):
    strs.append(','.join(groups[i]))
    print(strs[i])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [121]:
# loop and make batch apis calls
df = pd.DataFrame(columns=columns)
for string in strs:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
#WOW click the link
#     print(batch_api_url)
    data = requests.get(batch_api_url).json()
#     parse data and append them to the dataframe
    for symbol in string.split(','):
        df = df.append(pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            'n/a'
        ], index = columns)
               ,ignore_index=True     
                      )

In [122]:
df

# way faster that https requests!!!

,Ticker,Stocks Price,Market Cap,Number of Shares to buy
0,A,163,47792433400,n/a
1,AAL,18.77,11952830831,n/a
2,AAP,235.216,14960078505,n/a
3,AAPL,175.3,2978820710288,n/a
4,ABBV,138.46,241210271268,n/a
...,...,...,...,...
500,YUM,139.39,40130787087,n/a
501,ZBH,129.26,26280592578,n/a
502,ZBRA,603.837,32445282298,n/a
503,ZION,63.98,10103193193,n/a


### Calculating the number of shares to buy

In [123]:
portfolio_size= input('Enter the nmoney your portfolio?')

# Testing not good one
# make sure it is a integer
try:
    val = int(portfolio_size)
    print(val)
except:
    print(" That is not a number try again")
    portfolio_size= input('Enter the money for your portfolio?')


Enter the nmoney your portfolio?1000000
1000000


In [127]:
position_size = float(portfolio_size) / len(df)
for i in range(0, len(df.index)):
#     loc is an easy way to find your dat
    df.loc[i, 'Number of Shares to buy'] = math.floor(position_size / df.loc[i,'Stocks Price'])

In [128]:
df


,Ticker,Stocks Price,Market Cap,Number of Shares to buy
0,A,163,47792433400,12
1,AAL,18.77,11952830831,105
2,AAP,235.216,14960078505,8
3,AAPL,175.3,2978820710288,11
4,ABBV,138.46,241210271268,14
...,...,...,...,...
500,YUM,139.39,40130787087,14
501,ZBH,129.26,26280592578,15
502,ZBRA,603.837,32445282298,3
503,ZION,63.98,10103193193,30


### Saving our results to excell

In [135]:
df.to_csv('s&p_trades.csv')

### Formating